In [51]:
import pandas as pd

ad = pd.read_csv('datasets/1976-2020-president.csv')
years = set(ad['year'])
states = set(ad['state'])

In [52]:
def to_percent(older, col_key):
    newer = []
    for y in years:
        for s in states:
            try:
                d = older.loc[y, s, 'DEMOCRAT'][col_key]
                r = older.loc[y, s, 'REPUBLICAN'][col_key]
                newer.append([int(y), s, 100*d/(d+r)]) # % share for each state
            except KeyError as e:
                pass
    return pd.DataFrame(data=newer, columns=['year', 'state', col_key, ])

In [53]:
pop_vote = pd.read_csv('datasets/1976-2020-president.csv')
pop_vote = pop_vote[['year', 'state', 'party_simplified', 'candidatevotes', ]] # drop useless columns
pop_vote = pop_vote.groupby(['year', 'state', 'party_simplified', ]).sum() # drop multiple candidate
pop_vote = to_percent(pop_vote, 'candidatevotes')
pop_vote = pop_vote.rename(columns={"candidatevotes": "y_votes_percent", })
pop_vote.to_csv('datasets/pop_vote.csv', index=False)
pop_vote.head()

,year,state,y_votes_percent
0,1984,IOWA,46.276175
1,1984,NEW YORK,47.058282
2,1984,SOUTH DAKOTA,36.700487
3,1984,IDAHO,26.724429
4,1984,MONTANA,38.698601


In [ ]:
def splatter(a): # from row-column to table-list
    lst = []
    for year in a.columns[2:]:
        for index, row in a.iterrows():
            lst.append([int(year), row['GeoName'].upper(), row[year], ])
    return lst

In [54]:
a = pd.read_csv('datasets/gdp-nominal-97-20.csv')
b = pd.read_csv('datasets/gdp-nominal-77-97.csv')

nom_gdp = []

# from row-column to table-list

for year in a.columns[2:]:
    for index, row in a.iterrows():
        nom_gdp.append([int(year), row['GeoName'].upper(), row[year], ])

for year in b.columns[2:]:
    for index, row in b.iterrows():
        nom_gdp.append([int(year), row['GeoName'].upper(), row[year], ])

nom_gdp = [i for i in nom_gdp if i[1] != 'NEW ENGLAND'] # remove new england
nom_gdp = pd.DataFrame(data=nom_gdp, columns=['year', 'state', 'nominal_gdp_mln', ])
nom_gdp.to_csv('datasets/gdp-nominal.csv', index=False)
nom_gdp.head()

,year,state,nominal_gdp_mln
0,1997,ALABAMA,104811.9
1,1997,ALASKA,25810.8
2,1997,ARIZONA,132281.0
3,1997,ARKANSAS,59846.3
4,1997,CALIFORNIA,1071117.0


In [55]:
rgdp97 = pd.read_csv('datasets/gdp-real-77-97-chain-97.csv')
rgdp12 = pd.read_csv('datasets/gdp-real-97-20-chain-12.csv')



In [56]:
house_vote = pd.read_csv('datasets/1976-2020-house-utf8.csv')
house_vote = house_vote[['year', 'state', 'party', 'candidatevotes', ]]
house_vote = house_vote.groupby(['year', 'state', 'party', ]).sum()
house_vote = to_percent(house_vote, 'candidatevotes')
house_vote.head()
house_vote['year'] += 0
house_vote = house_vote.rename(columns={"candidatevotes": "houserep_votes_percent", })
house_vote.to_csv('datasets/house_vote.csv', index=False)
house_vote.head()

,year,state,houserep_votes_percent
0,1984,IOWA,46.922690
1,1984,NEW YORK,53.020269
2,1984,SOUTH DAKOTA,57.365079
3,1984,IDAHO,40.697895
4,1984,MONTANA,51.190297


In [57]:
df = pd.merge(pop_vote, nom_gdp, how='inner', left_on=['year','state'], right_on=['year','state'])
df = pd.merge(df, house_vote, how='inner', left_on=['year','state'], right_on=['year','state'])
df.head()

,year,state,y_votes_percent,nominal_gdp_mln,houserep_votes_percent
0,1984,IOWA,46.276175,41264.0,46.922690
1,1984,NEW YORK,47.058282,339427.1,53.020269
2,1984,SOUTH DAKOTA,36.700487,9194.0,57.365079
3,1984,IDAHO,26.724429,12585.9,40.697895
4,1984,MONTANA,38.698601,11010.2,51.190297


In [58]:
df.to_csv('datasets/xxx-final-dataset.csv', index=False)